In [4]:
import pickle
import pandas as pd
import numpy as np
from scipy.stats import kurtosis
from scipy.stats import skew
import scipy as sc

In [5]:
Train_df=pd.read_excel("C:\\Users\\shreyas.raghavan\\Documents\\Python Scripts\\Data.xlsx")

In [6]:
Train_df.head()

,Sales_Country,Disti_Country,DISTRIBUTOR_NAME,Partner_Name,PRODUCT_ID,SO_NUMBER,POS_TRANSACTION_ID,FISCAL_YEAR_QUARTER_NUMBER_INT,ADJUSTMENT_TYPE,BU,...,StandardMargin,ListPrice,COMP_US_NET_PRICE_AMOUNT,COMP_US_LIST_PRICE_AMOUNT,COMP_US_COST_AMOUNT,BOOK_QUANTITY,BOOK_NET,BOOK_LIST,BOOK_COST,EXTENDED_QUANTITY
0,AU,AU,WESTCON GROUP PTY LIMITED,TELSTRA CORPORATION LTD,CAB-ETH-S-RJ45=,307977997,308036529,20162,DSV Booking,SRGBU,...,82.12,20,7.8078,15.6,1.3962,1.0,10.01,20.0,1.79,0.78
1,AU,AU,INGRAM MICRO AUSTRALIA PTY LTD,J S R SYSTEMS SOLUTIONS,SLM2008PT-AU,309067137,309127522,20163,DSV Booking,SBTGBU,...,53.54,302,133.0100,302.0,61.7900,1.0,133.01,302.0,61.79,1.00
2,AU,AU,WESTCON GROUP PTY LIMITED,FUJITSU AUSTRALIA,C881G-4G-GA-K9,307174171,307122588,20162,DSV Booking,SRGBU,...,84.54,1800,144.7095,270.0,22.3665,1.0,964.73,1800.0,149.11,0.15
3,AU,AU,WESTCON GROUP PTY LIMITED,TELSTRA CORPORATION LTD,CAB-ETH-S-RJ45=,307754829,307857590,20162,DSV Booking,SRGBU,...,82.12,20,7.8078,15.6,1.3962,1.0,10.01,20.0,1.79,0.78
4,AU,AU,WESTCON GROUP PTY LIMITED,TELSTRA CORPORATION,CAB-ETH-S-RJ45=,317043891,317037133,20172,DSV Booking,SRGBU,...,81.54,20,2.3474,4.4,0.4334,1.0,10.67,20.0,1.97,0.22


In [7]:
features=list(Train_df.columns)
features.remove('PRODUCT_ID')
features.remove('POS_TRANSACTION_ID')
features.remove('SALES_COVERAGE_CODE')
features.remove('EXTENDED_QUANTITY')
features.remove('SO_NUMBER')


In [8]:
    def univariate(train):

        '''

            this function is for univariate analysis,

        '''

        #if 

        x = [x for x in train.columns if train[x].dtypes != 'object']

        data = train[x]

        #y = data.describe()

#         evcount = target.value_counts()

#         evratio = evcount[event]/target.count()

        def Range(y):

            mi = np.min(y)

            ma = np.max(y)

            rng = ma-mi

            return rng

        Rnge = data.apply(Range)

        def iqr(y):

            x25 = np.percentile(y,25)

            x75 = np.percentile(y,75)

            Iqr = x75 - x25

            return Iqr

           

        iQr = data.apply(iqr)

        def Skew(y):

            d = skew(y)

            return d

        skewness = data.apply(Skew)

        def Kurt(y):

            d = kurtosis(y)

            return d

        Kurtosis = data.apply(Kurt)

       

        mean = data.apply(np.mean)

        stdv = data.apply(np.std)

        median = data.apply(np.median)

        missingno = data.isnull().sum()

        def outlier(y):

            q1 = np.percentile(y,25)

            q3 = np.percentile(y,75)

            iqr = q3-q1

            outlier = ((y > (q3 +1.5*iqr)) |(y <(q1-1.5*iqr ))).sum()

            return outlier

        Q1 = data.quantile(.25) 
        
        p1=data.quantile(0.01)
        p3=data.quantile(0.99)

        Q3 = data.quantile(.75)

        out = data.apply(outlier)
         
        count=train.shape[0]
        
        missingper=(missingno/count)*100

        Frame = [Rnge,Q1,Q3,p1,p3,iQr,skewness,Kurtosis,mean,stdv,median,missingno,out,missingper]

        unistat = pd.concat(Frame,axis =1)

        unistat.columns = ['Range','Q1','Q3','p1','p99','IQR','Skewness','Kurtosis','mean','stdv','median','MissingValue','outlier','Missing_perc']

        l=[]
        
        for i in missingper:
            if i>99:
                l.append(1)
            else:
                l.append(0)
                
        unistat['Missing_Criteria']=l
        
        unistat=unistat[['MissingValue','Missing_perc','mean','median','stdv','p1','p99','Range','Q1','Q3','IQR','outlier','Skewness','Kurtosis']]
        z = [x for x in train.columns if train[x].dtypes == 'object']

        dat = train[z]

        cat = {}

        for i in z:

            cat[i]= dat[i].value_counts()
        
        data={'catdist':cat , 'unistat':unistat}
        k=0

        engine = 'xlsxwriter'

        writer = pd.ExcelWriter('univariates_bivariates_test.xlsx',engine=engine)

        uni=data['unistat']

        k=len(uni)+1

        uni.to_excel(writer,sheet_name='univariates',startrow=0,startcol=0,index=True)

        for i in data['catdist']:
          sum1=train.shape[0]
          report_df = pd.DataFrame()
          print(data['catdist'][i].index)
          report_df[i] = list(data['catdist'][i].index)
          report_df["Numbers"] = data['catdist'][i].values
          l1=[]
          for j in data['catdist'][i]:
              l1.append(j/sum1)
          report_df["perc"] = l1
#              print('reprt_df is')
#              print(report_df)
          report_df.to_excel(writer,sheet_name='univariates',startrow=k+1,startcol=0,index=True)

          k=len(data['catdist'][i])+2+k

        writer.close()

In [9]:
univariate(Train_df[features])

Index(['AU'], dtype='object')
Index(['AU ', 'NZ ', 'SG '], dtype='object')
Index(['WESTCON GROUP PTY LIMITED', 'INGRAM MICRO AUSTRALIA PTY LTD',
       'Dicker Data Limited Australia', 'INGRAM MICRO NZ LTD',
       'WESTCON GROUP NZ LTD', 'WESTCON GROUP PTE LTD'],
      dtype='object')
Index(['TELSTRA - HORIZON SALES AND DEVELOPMENT PTY LTD',
       'TELSTRA CORPORATION LTD', 'TELSTRA CORPORATION',
       'LEND LEASE BUILDING PTY LTD', 'MULTIMEDIA TECHNOLOGY PTY LTD',
       'UNKNOWN', 'DIMENSION DATA PTY LTD',
       'AUSTRALIA AND NEW ZEALAND BANKING GROUP LTD',
       'DATA 3 DISTRIBUTION AND LOGISTICS', 'BT AUSTRALASIA PTY LTD',
       ...
       'SATO AUSTRALIA PTY LTD', 'NATILIK', 'ZETTANET',
       'AT&T GLOBAL NETWORK SERVICES AUST PL',
       'GREEN TRIANGLE ELECTRONICS PTY LTD', 'OFFICE INFORMATION AUSTRAL',
       'ITECHICT PTY LTD', 'TECHWELL AUSTRALIA PTY LTD',
       'ALFA LAVAL AALBORG PTY LTD', 'ISCAPE PTY LTD'],
      dtype='object', length=1336)
Index(['DSV Booking   

In [10]:
inp_fea=features


In [11]:
inp_fea.remove('Sales_Country')
inp_fea.remove('Disti_Country')
inp_fea.remove('ListPrice')
inp_fea.remove('COMP_US_NET_PRICE_AMOUNT')
inp_fea.remove('COMP_US_LIST_PRICE_AMOUNT')
inp_fea.remove('COMP_US_COST_AMOUNT')

In [12]:
print(inp_fea)

['DISTRIBUTOR_NAME', 'Partner_Name', 'FISCAL_YEAR_QUARTER_NUMBER_INT', 'ADJUSTMENT_TYPE', 'BU', 'PF', 'L1_TECHNOLOGY_MKT_SEGMENT_NAME', 'L2_TECHNOLOGY_MKT_SEGMENT_NAME', 'L3_TECHNOLOGY_MKT_SEGMENT_NAME', 'BK_BUSINESS_ENTITY_NAME', 'PROMOTION_1_CD', 'Volume_Biz', 'Discount', 'StandardMargin', 'BOOK_QUANTITY', 'BOOK_NET', 'BOOK_LIST', 'BOOK_COST']


In [13]:
inp_fea.remove('Volume_Biz')
inp_fea.remove('BU')

In [14]:
cat_df=pd.get_dummies(Train_df[cat_col],drop_first=True)

NameError: name 'cat_col' is not defined

In [16]:
whole_df=pd.concat([Train_df,cat_df],axis=1)

NameError: name 'cat_df' is not defined

In [17]:
fea_in_model=[]

In [18]:
fea_in_model=list(cat_df.columns)
fea_in_model.extend(cont)

NameError: name 'cat_df' is not defined

In [538]:
fea_in_model

['Disti_Country_NZ ',
 'Disti_Country_SG ',
 'DISTRIBUTOR_NAME_INGRAM MICRO AUSTRALIA PTY LTD',
 'DISTRIBUTOR_NAME_INGRAM MICRO NZ LTD',
 'DISTRIBUTOR_NAME_WESTCON GROUP NZ LTD',
 'DISTRIBUTOR_NAME_WESTCON GROUP PTE LTD',
 'DISTRIBUTOR_NAME_WESTCON GROUP PTY LIMITED',
 'ADJUSTMENT_TYPE_ERP                           ',
 'ADJUSTMENT_TYPE_Regular POS                   ',
 'BU_ATSBU',
 'BU_CBABU',
 'BU_CMBU',
 'BU_CNGBU',
 'BU_CORBU',
 'BU_CPBU',
 'BU_CVEBU',
 'BU_CVIBU',
 'BU_DCSABU',
 'BU_EBBU',
 'BU_ECBU',
 'BU_ECMBU',
 'BU_EMAILBU',
 'BU_HCBU',
 'BU_IASBU',
 'BU_INSBU',
 'BU_IOTBU',
 'BU_MFGBU',
 'BU_NGFWBU',
 'BU_OPCBU',
 'BU_OPENDNSBU',
 'BU_PABU',
 'BU_PNABU',
 'BU_SAVBU',
 'BU_SBTGBU',
 'BU_SEBU',
 'BU_SRGBU',
 'BU_SSEBU',
 'BU_TMGBU',
 'BU_UABU',
 'BU_UCEBU',
 'BU_UCIBU',
 'BU_WEBBU',
 'BU_WEBXONGPLBU',
 'BU_WNBU',
 'L1_TECHNOLOGY_MKT_SEGMENT_NAME_Corp-Emerging Technologies',
 'L1_TECHNOLOGY_MKT_SEGMENT_NAME_Corp-Intelligent  Automation',
 'L1_TECHNOLOGY_MKT_SEGMENT_NAME_Corp-LAN 

In [19]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model

In [20]:
X_train, X_test, y_train, y_test = train_test_split(Train_df[inp_fea], Train_df['EXTENDED_QUANTITY'], test_size=0.50
                                                    , random_state=5)

In [21]:
lr=linear_model.LinearRegression()

In [22]:
lr.fit(X_train,y_train)

ValueError: could not convert string to float: 'PP-FAST-150124-32368'

In [544]:
preds=lr.predict(X_test)

In [545]:
from sklearn import metrics 

In [546]:
l=metrics.mean_squared_error(y_pred=preds,y_true=y_test)

In [547]:
print (l)

3.76945310256e+23
